# Exploratory Text Analysis

In [555]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('stopwords')
stopwords = stopwords.words('english')
remove_terms = ['said','the', '``', "''", "'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would']
stopwords = stopwords + remove_terms + list(string.punctuation)
print(stopwords)
lemmatizer = nltk.WordNetLemmatizer()

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carlostezna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
def preprocess(document):
        """
        Tokenizes documents then normalizes and lemmatizes tokens
        """
        from nltk.tokenize import word_tokenize
        words = word_tokenize(document)
        words_clean = []
        for word in words: # Go through every word in your tokens list
            w = word.lower()
            if (w not in stopwords):  # remove stopwords and punctuation
                words_clean.append(lemmatizer.lemmatize(w))
        return words_clean

In [11]:
def load_document(file):
    f = open(file)
    try:
        raw = f.read()
        return file.split('/')[-1], raw
    except:
        print(file)
        pass

def load_collection(files):
    texts = []
    for file in files:
        doc_id, text = load_document(file)
        texts.append(text)
    return texts

In [118]:
import os
COLLECTION_DIR = './dataset/newDataset/'
files = [COLLECTION_DIR + file for file in os.listdir(COLLECTION_DIR)]

corpus = load_collection(files)

# Feature Extraction

In [528]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# using default tokenizer in TfidfVectorizer
tfidf = TfidfVectorizer(token_pattern='(\S+)', min_df=3, max_df=0.8, 
                        ngram_range=(1, 2), stop_words=stopwords, tokenizer=word_tokenize)
features = tfidf.fit_transform(corpus)
df = pd.DataFrame(
        features.todense(),
        columns=tfidf.get_feature_names()
    )

In [529]:
df.head()

,'best,'bot,'bot nets,'cold,'cold calls,'could,'do,'eu,'goodbye,'goodbye said,...,£8.5bn,£800m,£800m 1.5bn,£80m,£857m,£8bn,£8m,£9.4m,£99,£9m
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [530]:
len(tfidf.vocabulary_)

31809

In [764]:
len(corpus)

2135

In [340]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
doc_set = [preprocess(doc) for doc in corpus]
finder = BigramCollocationFinder.from_words(doc_set[0])
print(finder.nbest(bigram_measures.pmi, 10))
#finder.apply_freq_filter(10)

[('16-year-old', 'depressed'), ('1990', '2004'), ('1999', '2003'), ('2', 'european'), ('2004', '25'), ('address', 'issue'), ('alerted', 'experience'), ('also', 'highlighted'), ('amount', 'serious'), ('appearance', 'series')]


In [341]:
word_fd = nltk.FreqDist(doc_set[0])
bigram_fd = nltk.FreqDist(nltk.bigrams(doc_set[0]))
finder = BigramCollocationFinder(word_fd, bigram_fd)
finder.score_ngrams(bigram_measures.raw_freq)[0:10]

[(('vulnerable', 'people'), 0.014705882352941176),
 (('taking', 'life'), 0.011029411764705883),
 (('alcohol', 'problem'), 0.007352941176470588),
 (('bar', 'cell'), 0.007352941176470588),
 (('custody', 'death'), 0.007352941176470588),
 (('death', 'custody'), 0.007352941176470588),
 (('death', 'rate'), 0.007352941176470588),
 (('drug', 'alcohol'), 0.007352941176470588),
 (('highly', 'vulnerable'), 0.007352941176470588),
 (('human', 'right'), 0.007352941176470588)]

In [348]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(doc_set[0])
finder.score_ngrams(trigram_measures.raw_freq)[0:10]

[(('drug', 'alcohol', 'problem'), 0.007352941176470588),
 (('life', 'highly', 'vulnerable'), 0.007352941176470588),
 (("'shocks", 'mp', 'death'), 0.003676470588235294),
 (('16-year-old', 'depressed', 'exhibiting'), 0.003676470588235294),
 (('1990', '2004', '25'), 0.003676470588235294),
 (('1999', '2003', 'mp'), 0.003676470588235294),
 (('2', 'european', 'convention'), 0.003676470588235294),
 (('2002', 'urged', 'home'), 0.003676470588235294),
 (('2003', 'mp', 'said'), 0.003676470588235294),
 (('2004', '25', 'child'), 0.003676470588235294)]

# Topic Modeling

In [556]:
import gensim
from gensim import corpora, models
dictionary = corpora.Dictionary(preprocess(doc) for doc in corpus)
bow = [dictionary.doc2bow(preprocess(doc)) for doc in corpus]

In [557]:
import time
start = time.time()
ldamodel = models.ldamodel.LdaModel(bow, num_topics=20, id2word=dictionary, passes=50, minimum_probability=0.1)
end = time.time()
print('Train time: ', end - start)

Train time:  97.81347012519836


In [763]:
topics = ldamodel.print_topics(num_topics=20, num_words=20)
for t in topics:
    print(t)
    print()

(0, '0.022*"mr" + 0.010*"people" + 0.010*"school" + 0.009*"also" + 0.007*"say" + 0.007*"eu" + 0.007*"campaign" + 0.006*"country" + 0.005*"international" + 0.005*"east" + 0.005*"government" + 0.005*"year" + 0.005*"ukip" + 0.005*"uk" + 0.005*"plan" + 0.004*"north" + 0.004*"bell" + 0.004*"area" + 0.004*"health" + 0.004*"member"')

(1, '0.023*"phone" + 0.016*"mobile" + 0.013*"people" + 0.013*"camera" + 0.011*"firm" + 0.009*"service" + 0.007*"message" + 0.007*"consumer" + 0.007*"multimedia" + 0.006*"customer" + 0.006*"new" + 0.006*"get" + 0.006*"mr" + 0.006*"image" + 0.005*"sold" + 0.005*"month" + 0.005*"way" + 0.005*"japan" + 0.005*"number" + 0.005*"vodafone"')

(2, '0.030*"police" + 0.011*"power" + 0.010*"men" + 0.009*"officer" + 0.009*"csos" + 0.009*"home" + 0.008*"plan" + 0.008*"offence" + 0.007*"authority" + 0.007*"policing" + 0.007*"wale" + 0.007*"act" + 0.007*"community" + 0.006*"house" + 0.006*"castaignede" + 0.006*"common" + 0.006*"hunting" + 0.006*"chamber" + 0.006*"hunt" + 0.006*

In [572]:
ldamodel.show_topic(7, topn=20)

[('mr', 0.033849884),
 ('labour', 0.016744204),
 ('election', 0.015839843),
 ('party', 0.015327615),
 ('tory', 0.01469126),
 ('blair', 0.011385909),
 ('say', 0.011297151),
 ('howard', 0.008468125),
 ('minister', 0.008309312),
 ('people', 0.0075308434),
 ('leader', 0.006769998),
 ('campaign', 0.0067287604),
 ('prime', 0.0066819917),
 ('make', 0.0060810056),
 ('mp', 0.005895772),
 ('student', 0.005760478),
 ('government', 0.00574761),
 ('told', 0.005530442),
 ('lib', 0.0054735737),
 ('britain', 0.0053167245)]

In [569]:
doc_bow = dictionary.doc2bow(preprocess(corpus[0]))
ldamodel.get_document_topics(doc_bow)

[(7, 0.11320306), (13, 0.123926796), (17, 0.3080736), (19, 0.22229713)]

# Word Embeddings

In [115]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', 
                                                                binary = True)

In [765]:
analogy = wv_embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
print(analogy)

[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431607246399), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454)]


In [545]:
similar_word = wv_embeddings.most_similar('shit')
print(similar_word)

[('sh_*_t', 0.8806686401367188), ('crap', 0.8299816250801086), ('fucking', 0.8001799583435059), ('sh_t', 0.7744057178497314), ('fuckin', 0.7668500542640686), ('sh_**', 0.7638066411018372), ('fuck', 0.7604621648788452), ('f_*_cking', 0.7250384092330933), ("y'know", 0.7228368520736694), ('shyt', 0.7212573885917664)]


# Word vectors

In [581]:
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases

In [582]:
# Cleaning data - remove punctuation from every text
texts = corpus.copy()
sentences = []
# Go through each text in turn
for ii in range(len(texts)):
    sentences = [re.sub(pattern=r'[\!"#$%&\*+,-./:;<=>?@^_`()|~=]', 
                        repl='', 
                        string=x
                       ).strip().split(' ') for x in texts[ii].split('\n') 
                      if not x.endswith('writes:')]
    sentences = [x for x in sentences if x != ['']]
    texts[ii] = sentences

In [583]:
# concatenate all sentences from all texts into a single list of sentences
all_sentences = []
for text in texts:
    all_sentences += text

In [584]:
# Phrase Detection
# Give some common terms that can be ignored in phrase detection
# For example, 'state_of_affairs' will be detected because 'of' is provided here: 
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
# Create the relevant phrases from the list of sentences:
phrases = Phrases(all_sentences, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)
# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_sentences])

In [585]:
import time
start = time.time()
model = Word2Vec(all_sentences, 
                 min_count=2,   # Ignore words that appear less than this
                 size=200,      # Dimensionality of word embeddings
                 workers=8,     # Number of processors (parallelisation)
                 window=5,      # Context window for words during training
                 iter=100)       # Number of epochs training over corpus
end = time.time()
print('Train time: ', end - start)

Train time:  56.32094478607178


In [766]:
model.wv.most_similar('Google') # == model.wv.similar_by_word('Apple')

[('Yahoo', 0.48415762186050415),
 ('search_engine', 0.45970356464385986),
 ('site', 0.4212949275970459),
 ('web', 0.42027774453163147),
 ('search', 0.41543716192245483),
 ('Microsoft', 0.3831966519355774),
 ('users', 0.3824462294578552),
 ('MSN', 0.3818070888519287),
 ('Ask_Jeeves', 0.36329710483551025),
 ('AOL', 0.3607094883918762)]

In [756]:
model.wv.distance('man', 'dog')

0.9098354652523994

In [598]:
model.wv.similarity('Apple', 'Google')

0.33836627

In [600]:
model.wv.closer_than('Apple', 'Google') # returns entities that are more similar to Apple than Google

['software',
 'Microsoft',
 'iPod',
 'Nintendo',
 'browser',
 'iTunes',
 'search_engine',
 'operating_system',
 'Mac_mini',
 'Motorola',
 'IPTV',
 'Internet_Explorer']

# FastText Algorithm

In [639]:
from gensim.models.fasttext import FastText
import time
start = time.time()
fasttext_model = FastText(all_sentences, 
                 min_count=2,   # Ignore words that appear less than this
                 size=200,      # Dimensionality of word embeddings
                 workers=8,     # Number of processors (parallelisation)
                 window=5,      # Context window for words during training
                 iter=100) 
end = time.time()
print('Train time: ', end - start)

Train time:  338.56731486320496


In [757]:
fasttext_model.wv.most_similar('Apple')

[('Apple_iPod', 0.7879966497421265),
 ("Apple's_iPod", 0.727531373500824),
 ('Applegate', 0.7225794792175293),
 ('AppleInsidercom', 0.6945337653160095),
 ("Apple's_iTunes", 0.6681185960769653),
 ('cripple', 0.6454294919967651),
 ("Apple's", 0.6317760944366455),
 ('Hoople', 0.5639526844024658),
 ('Temple', 0.4836748540401459),
 ('purple', 0.4782470166683197)]

# Text Summarization

In [714]:
sentences = nltk.sent_tokenize(corpus[0])
total_documents = len(sentences)

In [715]:
def sentence_frequency_matrix(sentences, stopWords):
    frequency_matrix = {}
    lem = lemmatizer

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = lem.lemmatize(word)
            if word in stopWords:
                continue
    
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [716]:
freq_matrix = sentence_frequency_matrix(sentences, stopwords)

In [717]:
def sentence_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [718]:
tf_matrix = sentence_tf_matrix(freq_matrix)

In [719]:
def sentence_df_matrix(freq_matrix):
    df_matrix = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in df_matrix:
                df_matrix[word] += 1
            else:
                df_matrix[word] = 1

    return df_matrix

In [720]:
df_matrix = sentence_df_matrix(freq_matrix)

In [721]:
def sentence_idf_matrix(freq_matrix, df_matrix, total_documents):
    import math
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(df_matrix[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [722]:
idf_matrix = sentence_idf_matrix(freq_matrix, df_matrix, total_documents)

In [723]:
def sentence_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, tf_table), (sent2, idf_table) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(tf_table.items(), 
                                                    idf_table.items()):  # keys are same: word1 == word2
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [724]:
tf_idf_matrix = sentence_tf_idf_matrix(tf_matrix, idf_matrix)

In [725]:
def score_sentences(tf_idf_matrix):
    sentence_scores = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentence_scores[sent] = total_score_per_sentence / count_words_in_sentence

    return sentence_scores

In [726]:
sentence_scores = score_sentences(tf_idf_matrix)

In [727]:
print(sentence_scores)

{'Custody death r': 0.10179248221200314, 'The joint commi': 0.06977472553820856, 'Members urged t': 0.07481870532259614, 'There was one p': 0.10232148706256168, 'The report, whi': 0.056499799137089766, 'Many of those w': 0.09976876201152865, 'It questioned w': 0.10289222174044234, 'Increased resou': 0.10025186589522289, 'Committee chair': 0.0918899215712234, '"Yet throughout': 0.05409358357349983, '"These highly v': 0.09649459673957755, '"Crime levels a': 0.14957262692375967, 'The misplaced o': 0.10152967460208544, '"Until we chang': 0.06723165486153318, 'The committee a': 0.0888845243928637, 'Between 1990 an': 0.09034930525444314, 'It picked out t': 0.058903018563800966, 'It revealed tha': 0.06826391952557925, 'Even though the': 0.07132881808537661, 'Nine days into ': 0.12751924344770646}


In [728]:
def calc_average_score(sentence_scores):
    sumValues = 0
    for entry in sentence_scores:
        sumValues += sentence_scores[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentence_scores))

    return average

In [729]:
threshold = calc_average_score(sentence_scores)

In [730]:
threshold

0.08870904682305511

In [731]:
def generate_summary(sentences, sentence_scores, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentence_scores and sentence_scores[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [732]:
original_text = generate_summary(sentences, sentence_scores, 0.0 * threshold)
print(original_text)

 Custody death rate 'shocks' MPs

Deaths in custody have reached "shocking" levels, a committee of MPs and peers has warned. The joint committee on human rights found those committing suicide were mainly the most vulnerable, with mental health, drugs or alcohol problems. Members urged the government to set up a task force to tackle deaths in prisons, police cells, detention centres and special hospitals. There was one prison suicide every four days between 1999 and 2003, MPs said. The report, which followed a year-long inquiry by the committee, found the high death rate "amounts to a serious failure to protect the right to life of a highly vulnerable group". Many of those who ended up taking their own lives had "presented themselves" to the authorities with these problems before they even offended, the report said. It questioned whether prison was the most appropriate place for them to be kept and whether earlier intervention would have meant custody could have been avoided. Increased 

In [771]:
summary = generate_summary(sentences, sentence_scores, 1.2 * threshold)
print((summary))

 "Crime levels are falling but we are holding more people in custody than ever before. Nine days into his sentence, Joseph hung himself from the bars of his cell window with a sheet.


### Using Gensim

In [655]:
from gensim.summarization import keywords
doc_keywords = keywords(corpus[0], lemmatize=True, split=True, scores=True)

In [660]:
[print(d) for d in doc_keywords];

('vulnerable', 0.2368033373815465)
('deaths', 0.20470307106154545)
('suicidal', 0.1875362084047901)
('joseph', 0.1762635431929641)
('ill', 0.17352340373031905)
('said', 0.16982955916691264)
('shocking', 0.16123479880996985)
('training', 0.14716448411524904)
('young', 0.13966305405441834)
('custody', 0.1389087974944819)
('people', 0.1387280465544515)
('committee', 0.13763098003321853)
('year', 0.13560464463693003)
('increased', 0.1340595962312684)
('cell', 0.1322005806232812)
('detention', 0.12873521419305609)
('rate', 0.12454690603717374)
('problems', 0.11925119378856822)
('worrying', 0.11925119378856822)
('members', 0.11925119378856808)


In [681]:
from gensim.summarization import mz_entropy
mz_keys = mz_entropy.mz_keywords(corpus[0], blocksize=256, split=True, scores=True)

In [683]:
[print(d) for d in mz_keys];

('his', 0.010513140658998924)
('death', 0.006361288459325908)
('said', 0.006361288459325908)
('mps', 0.004232401509370351)
('report', 0.004232401509370351)
('two', 0.004232401509370351)
('custody', 0.0024383279677150224)
('alcohol', 0.002067209604442253)
('bars', 0.002067209604442253)
('cell', 0.002067209604442253)
('children', 0.002067209604442253)
('deaths', 0.002067209604442253)
('every', 0.002067209604442253)
('found', 0.002067209604442253)
('from', 0.002067209604442253)
('highly', 0.002067209604442253)
('himself', 0.002067209604442253)
('joseph', 0.002067209604442253)
('many', 0.002067209604442253)
('rate', 0.002067209604442253)
('suicide', 0.002067209604442253)
('these', 0.002067209604442253)
('those', 0.002067209604442253)
('time', 0.002067209604442253)
('training', 0.002067209604442253)
('up', 0.002067209604442253)
('whether', 0.002067209604442253)
('young', 0.002067209604442253)
('committee', 0.0009774089866891895)
('we', 0.0009774089866891895)


In [687]:
from gensim.summarization.keywords import get_graph
from gensim.summarization.pagerank_weighted import pagerank_weighted
doc_graph = get_graph(corpus[0])

In [688]:
result = pagerank_weighted(doc_graph)

In [693]:
result

{'custodi': 0.13750126310987576,
 'death': 0.2084065958635447,
 'rate': 0.12592202234184363,
 'shock': 0.16470581891416153,
 'mp': 0.1222347362139448,
 'reach': 0.04655278762216555,
 'level': 0.08862783810546443,
 'committe': 0.13774437018197536,
 'peer': 0.00980969127764677,
 'warn': 0.009809691277646748,
 'joint': 0.05078095910434941,
 'human': 0.09111113377406833,
 'right': 0.09111113377406845,
 'commit': 0.0518635066227884,
 'suicid': 0.1885118235529371,
 'mainli': 0.009809691277646733,
 'vulner': 0.23868637211919252,
 'mental': 0.08845007253021196,
 'health': 0.08873658185340709,
 'drug': 0.08845007253021193,
 'alcohol': 0.06300183536572451,
 'problem': 0.11922043218241044,
 'member': 0.1192204321824106,
 'urg': 0.06300183536572466,
 'govern': 0.009809691277646793,
 'set': 0.009809691277646776,
 'task': 0.09111113377406838,
 'forc': 0.09111113377406851,
 'tackl': 0.04700329466642917,
 'prison': 0.09235975254010181,
 'polic': 0.09076490566037149,
 'cell': 0.13362976231685467,
 'det

In [710]:
from gensim.summarization.summarizer import summarize, summarize_corpus

In [709]:
summarize(corpus[0], ratio=0.5)

'Deaths in custody have reached "shocking" levels, a committee of MPs and peers has warned.\nThe joint committee on human rights found those committing suicide were mainly the most vulnerable, with mental health, drugs or alcohol problems.\nMembers urged the government to set up a task force to tackle deaths in prisons, police cells, detention centres and special hospitals.\nThere was one prison suicide every four days between 1999 and 2003, MPs said.\nThe report, which followed a year-long inquiry by the committee, found the high death rate "amounts to a serious failure to protect the right to life of a highly vulnerable group".\nMany of those who ended up taking their own lives had "presented themselves" to the authorities with these problems before they even offended, the report said.\n"Yet throughout our inquiry we have seen time and time again that extremely vulnerable people are entering custody with a history of mental illness, drug and alcohol problems and potential for taking 